In [2]:
import tensorflow as tf
import os
from PIL import Image
import numpy
import glob

In [3]:
# Change to your final model.
loaded_raw = tf.saved_model.load("results/FinalTuned")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [4]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0', '/device:GPU:0']


In [24]:
labels = ["up","down","left","right"]
total_correct = 0
total = 0
total_batch = 0
EPOCHS=30*20
BATCH_SIZE=32
for counter in range(4):
    # Change to train for training data.
    path = "./processed3/validate/"+labels[counter]
    matrix = {}
    x=0
    for _ in range(EPOCHS):
        examples = []
        for filename in glob.glob(os.path.join(path, '*.jpg')): 
            image =  Image.open(filename)
            example=numpy.array(image)
            examples.append(example)
            x+=1
            if x%BATCH_SIZE==0:
                break    
        test=tf.convert_to_tensor(examples,tf.float32)
        results = loaded_raw.signatures["predict"](input=test)["classes"]
        unique, counts = numpy.unique(results.numpy(), return_counts=True)
        batch = dict(zip(unique, counts))
        for key in batch:
            if key in matrix:
                matrix[key]= batch[key] + matrix[key]
            else:
                matrix[key]= batch[key]
    print(labels[counter])
    print(matrix)
    total_correct += matrix[counter+1]
    total+=BATCH_SIZE*EPOCHS
    print(matrix[counter+1]/(BATCH_SIZE*EPOCHS))
print(total_correct/total)    

up
{1: 18600, 2: 600}
0.96875
down
{2: 19200}
1.0
left
{3: 18000, 4: 1200}
0.9375
right
{3: 3600, 4: 15600}
0.8125
0.9296875
